# Calculate AUCs for bootstraps using STS model scores

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

## Load y and y_hat for entire STS DataFrame

In [ ]:
root = os.path.expanduser("~/dropbox/sts-data")
fpath = os.path.join(root, "sts-mgh.csv")
df = pd.read_csv(fpath, low_memory=False)

## Iterate over bootstrap test sets

In [ ]:
path_to_bootstraps = os.path.expanduser("~/dropbox/sts-data")
cohorts = [
#     "cabg",
#     "valve",
#     "cabg-valve",
    "major",
#     "other",
#     "all-elective",
    "major-elective",
#     "other-elective",
]

In [ ]:
aucs = {}
for cohort in cohorts:
    aucs[cohort] = []
    for bootstrap in range(10):
        fpath = os.path.join(path_to_bootstraps, f"bootstraps-ecg-{cohort}", str(bootstrap), "test.csv")
        test = pd.read_csv(fpath, low_memory=False, usecols=["mrn", "death", "predmort"])

        y = test.death.to_numpy()
        y_hat = test.predmort.to_numpy()
        n_tot = len(y)

        n_scores = sum(~np.isnan(y_hat))
        frac_scores = n_scores / len(y) * 100

        not_nan = ~np.isnan(y_hat)
        y = y[not_nan]
        y_hat = y_hat[not_nan]

        auc = roc_auc_score(y_true=y, y_score=y_hat)
        aucs[cohort].append(auc)

        print(f"{cohort}: bootstrap {bootstrap+1}, {n_scores}/{n_tot} patients have STS scores ({frac_scores:.1f}%), AUC={auc:0.3f}")

In [ ]:
df_aucs = pd.DataFrame(aucs)
df_aucs.index.name = "bootstrap"
df_aucs

In [ ]:
fpath = os.path.expanduser("~/dropbox/sts-net/figures-and-tables/sts.csv")
df_aucs.to_csv(fpath)
print(f"Saved AUCs of STS risk score predictions to {fpath}")